In [23]:
import os 
from dotenv import load_dotenv
from typing import List
from langchain_groq import ChatGroq
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [4]:
load_dotenv()

True

In [6]:
groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [ ]:
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN") 

In [8]:
llm = ChatGroq(groq_api_key=groq_api_key, model="Llama3-8b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000023A6D3AF6A0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000023A6D3AD240>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [2]:
documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]

In [10]:
hf_embeddings = HuggingFaceEmbeddings(model="all-MiniLM-L6-v2")
hf_embeddings

h:\Data Scientists Projects\LLM-Powered Chatbot with Message History using Langchain\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [11]:
vector_store = Chroma.from_documents(documents=documents, embedding= hf_embeddings)
vector_store

In [12]:
vector_store.similarity_search("cat")

[Document(id='701741c0-6b2a-4c14-94a1-a749bba52df5', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='7f0b58cd-0250-43b2-9f5e-49fda04a5c99', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='bd7d35c0-8a42-4987-87ea-e4ccfedb2af2', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(id='59508c25-7d2f-4bdb-9fca-72f85c44c1ad', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [13]:
await vector_store.asimilarity_search("cat")

[Document(id='701741c0-6b2a-4c14-94a1-a749bba52df5', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='7f0b58cd-0250-43b2-9f5e-49fda04a5c99', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='bd7d35c0-8a42-4987-87ea-e4ccfedb2af2', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(id='59508c25-7d2f-4bdb-9fca-72f85c44c1ad', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [14]:
vector_store.similarity_search_with_score("cat")

[(Document(id='701741c0-6b2a-4c14-94a1-a749bba52df5', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.9351056218147278),
 (Document(id='7f0b58cd-0250-43b2-9f5e-49fda04a5c99', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  1.5740898847579956),
 (Document(id='bd7d35c0-8a42-4987-87ea-e4ccfedb2af2', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
  1.595690369606018),
 (Document(id='59508c25-7d2f-4bdb-9fca-72f85c44c1ad', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
  1.665792465209961)]

In [19]:
first_retriever = RunnableLambda(vector_store.similarity_search).bind(k=1)
first_retriever

RunnableBinding(bound=RunnableLambda(similarity_search), kwargs={'k': 1}, config={}, config_factories=[])

In [20]:
first_retriever.batch(["cat", "dog"])

[[Document(id='701741c0-6b2a-4c14-94a1-a749bba52df5', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='7f0b58cd-0250-43b2-9f5e-49fda04a5c99', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [21]:
second_retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1}
)
second_retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x0000023A0C75C9A0>, search_kwargs={'k': 1})

In [22]:
second_retriever.batch(["cat", "dog"])

[[Document(id='701741c0-6b2a-4c14-94a1-a749bba52df5', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='7f0b58cd-0250-43b2-9f5e-49fda04a5c99', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [24]:
message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""
message

'\nAnswer this question using the provided context only.\n\n{question}\n\nContext:\n{context}\n'

In [25]:
prompt = ChatPromptTemplate.from_messages([("human", message)])
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nAnswer this question using the provided context only.\n\n{question}\n\nContext:\n{context}\n'), additional_kwargs={})])

In [26]:
rag_chain = {"context": second_retriever, "question": RunnablePassthrough()} | prompt | llm
rag_chain

{
  context: VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x0000023A0C75C9A0>, search_kwargs={'k': 1}),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nAnswer this question using the provided context only.\n\n{question}\n\nContext:\n{context}\n'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000023A6D3AF6A0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000023A6D3AD240>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [27]:
response = rag_chain.invoke("Tell me about dogs!")
response

AIMessage(content='According to the provided context, dogs are great companions known for their loyalty and friendliness.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 88, 'total_tokens': 107, 'completion_time': 0.016930568, 'prompt_time': 0.010501345, 'queue_time': 0.142494818, 'total_time': 0.027431913}, 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_0fb809dba3', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--19714322-5bdd-47b9-bdec-147d28da5d44-0', usage_metadata={'input_tokens': 88, 'output_tokens': 19, 'total_tokens': 107})

In [28]:
response.content

'According to the provided context, dogs are great companions known for their loyalty and friendliness.'